In [0]:
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install
! git clone https://github.com/neheller/kits19.git

In [0]:
%cd kits19
! python -m starter_code.get_imaging

In [0]:
%cd kits19/
from starter_code.visualize import visualize

for i in range(300):
  visualize("case_0000"+i,"/content/kits19/visualized/case_00003")



Komórka nie była używana 


In [0]:
import scipy
import matplotlib.pyplot as plt
from skimage import  color

im = plt.imread('/content/kits19/visualized/case_00002/00100.png') 
im_gray = color.rgb2gray(im)
for x in range(im.shape[0]):
  for y in range(im.shape[1]):
    if(im[x,y,0]<=0.60):
      im[x,y,0]=0

gauss = gaussian_smoothing(im_gray,3.0)

plt.subplot(1,3,1)
plt.imshow(im_gray,cmap='gray')
plt.title("gray")
plt.subplot(1,3,2)
plt.imshow(gauss,cmap='gray')
plt.title('gauss')
plt.subplot(1,3,3)
plt.imshow(im,cmap='gray')
plt.title('with red')
plt.show()

W tej komórce otwierane są otrzymane dane ze zbioru Kits2019 za pomocą funkcji już przygotowanych do ich obsługi. Proces wykonywany jest dla wszystkich zdjęć automatycznie. Proces został sprawdzany manulanie jedynie dla folderów, które mogły zostać obsłużone przez pamięć obliczeniową o małym ramie. Przedstawiony proecs automatyczny poniżej zostaje zatrzymany podczas wczytania przypadku, w którym zabrakło pamięci obliczeniowej, na podstawie czego uważamy, że funckja działa poprawnie.


In [0]:
import matplotlib.pyplot as plt
from starter_code.utils import load_case
from imageio import imwrite
from pathlib import Path
import argparse

import scipy.misc
import numpy as np

for i in range(300):
  i+=1
  volume, segemnatation=load_case('case_0000'+str(i))

  mask_out_path = Path("/content/kits19/visualized/case_0000"+str(i)+"/masks_0000"+str(i))
  image_out_path = Path("/content/kits19/visualized/case_0000"+str(i)+"/images_0000"+str(i))
  s=segemnatation.get_fdata()
  v=volume.get_fdata()

  for j in range(s.shape[0]):  
    maskfpath = mask_out_path / ("{:05d}.png".format(j))
    imagefpath = image_out_path / ("{:05d}.png".format(j))
    imwrite(str(maskfpath), s[j,:,:])
    imwrite(str(imagefpath), v[j,:,:])



Nieużywana funkcja do zmiany rozmiarów obrazków, jeśli sieć będzie za płytka, a rozmiar zdjęć nie bedzie pozwalał na prace na posiadanym RAM'ie. Której cos brakuje 


In [0]:
import os 
from skimage.io import imread, imsave
import cv2


path = "/content/kits19/visualized/case_00002/images_00002"
scale = 128/512
for image_path in os.listdir(path):

  input_path = os.path.join(path, image_path)
  img = imread(input_path)

  width = int(img.shape[1] * scale)
  hight = int(img.shape[0] * scale)
  dim=(width,hight)
  resized = cv2.resize(img, dim,interpolation=cv2.INTER_AREA)

  fullpath = os.path.join(path, image_path)
  imsave(fullpath, resized)

W poniższym oknie tworzono architekturę sieci. Zamiarem było stworzyć architekturę AlexNet. Jednakże początkwo wprowadzono jedną z prostszych architektura, by sprawdzić działanie pracy. Jednakże ze względu na błąd, który prawdopodobnie wynika ze źle przygotowanych danych, nie jest możliwe aby sprwadzić działanie architektury.

In [0]:
from keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
import matplotlib.pyplot as plt 
import os
import numpy as np
from tqdm import tqdm
from skimage.io import imread

TRAIN_PATH = "/content/kits19/visualized/case_00002/"
TEST_PATH = '/content/kits19/visualized/case_00003/images_00003/'

train_ids = next(os.walk(TRAIN_PATH))[2]
test_ids = next(os.walk(TEST_PATH))[1]

X_train = np.zeros((len(train_ids),512,512,3),dtype=np.uint8)
Y_train = np.zeros((len(train_ids),512,512,1),dtype=np.bool)

X_test = np.zeros((len(test_ids),512,512,3),dtype=np.uint8)

for n, id_ in tqdm(enumerate(train_ids),total=len(train_ids)):
  path = TRAIN_PATH+id_
  img=imread(path+'/images_00002/'+id_+'.png')[:,:,:,3]
  X_train[n] = img

  for mask_file in next(os.walk(path+'/masks_00002/'))[2]:
    mask_=imread(path+'/masks_00002/'+mask_file)
    mask = np.maximum(Y_train,mask_)

  Y_train[n]=mask

for n, id_ in tqdm(enumerate(test_ids),total=len(test_ids)):
  path = TEST_PATH+id_
  img=imread(path+'/images_00003/'+id_+'.png')[:,:,:,3]
  X_test[n]=img

model = Sequential()

model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(512, 512, 3)))
model.add(Dropout(0.4))
model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(50, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(10, activation='softmax'))

model.summary()

0it [00:00, ?it/s]
0it [00:00, ?it/s]


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 512, 512, 50)      1400      
_________________________________________________________________
dropout_8 (Dropout)          (None, 512, 512, 50)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 512, 512, 125)     56375     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 256, 256, 125)     0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 256, 256, 125)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192000)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)               

Trenowanie danych ale coś nie gra :/


In [0]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [0]:
import tensorflow as tf

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]

results=model.fit(X_train, Y_train,validation_split=0.1, batch_size=16, epochs=25,callbacks=callbacks)

Funkacja do ewaluacji - wskaźnik Jaccard index.


In [0]:
import numpy as np
def evaluation(target,prediction):
  score=[]
  for i in range (target.shape[0]):
     intersection = np.logical_and(target[i,:,:], prediction[i,:,:])
     union = np.logical_or(target[i,:,:], prediction[i,:,:])
     if np.sum(union)!=0:
       jac = np.sum(intersection)/np.sum(union)
       score.append(jac)
  return np.mean(score)

score= evaluation(s,s)